We use collab here to merge the processed compustat dataset and the returns dataset using merge_asof.
Either run on collab or have enough ram to handle the datasets.

In [ ]:
# Run using Collab
from google.colab import drive
import os
import pandas as pd
drive.mount('/content/gdrive',force_remount=True)
folder = '/content/gdrive/My Drive/datasets_mlfin'

Mounted at /content/gdrive


In [ ]:
#Run Locally
folder = "datasets/PostProcessed/"

In [ ]:
returns_df = pd.read_csv(os.path.join(folder, 'processed_returns.csv'))
compustat_df = pd.read_csv(os.path.join(folder, 'processed_data_compustat.csv'))

<ipython-input-10-1833506057>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  returns_df = pd.read_csv(os.path.join(folder, 'processed_returns.csv'))


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# 2. Standardize CUSIP format
returns_df = returns_df.rename(columns={'CUSIP': 'cusip'})
returns_df['cusip'] = returns_df['cusip'].astype(str).str.strip().str[:8]
compustat_df['cusip'] = compustat_df['cusip'].astype(str).str.strip().str[:8]

# 3. Convert dates to datetime
returns_df['MthCalDt'] = pd.to_datetime(returns_df['MthCalDt'])
compustat_df['datadate'] = pd.to_datetime(compustat_df['datadate'])

# 5. Print some diagnostic information
print("Returns dataset shape:", returns_df.shape)
print("Compustat dataset shape:", compustat_df.shape)
print("Returns date range:", returns_df['MthCalDt'].min(), "to", returns_df['MthCalDt'].max())
print("Compustat date range:", compustat_df['datadate'].min(), "to", compustat_df['datadate'].max())
print("Common CUSIPs:", len(set(returns_df['cusip']) & set(compustat_df['cusip'])))

# 6. Perform the asof merge
# This will match each return with the most recent compustat data available before the return date
merged_df = pd.merge_asof(
    returns_df.sort_values('MthCalDt'),
    compustat_df.sort_values('datadate'),
    left_on='MthCalDt',
    right_on='datadate',
    by='cusip',
    direction='backward',
    tolerance=pd.Timedelta('7 days')  # Allow matching within the last 90 days
)

# 7. Check the merge results
print("\nMerged dataset shape:", merged_df.shape)
print("Merged dataset unique CUSIPs:", merged_df['cusip'].nunique())

# 8. Identify rows with Compustat data
merged_df['has_compustat'] = merged_df['datadate'].notna()
print("Rows with Compustat data:", merged_df['has_compustat'].sum())
print("Percentage with Compustat data:", merged_df['has_compustat'].mean() * 100, "%")

# 9. Define the predictors we want to use
predictors = [col for col in [
    'epspxy', 'oiadpy', 'saley', 'earnings_growth',
    'revenue_growth', 'eps_surprise', 'dividend_change',
    'repurchase_intensity'
] if col in merged_df.columns]

# 10. Handle missing values more intelligently
# First, ensure we have returns
merged_df = merged_df.dropna(subset=['MthRet'])

# Check missing value percentages for each predictor
missing_percentages = merged_df[predictors].isna().mean() * 100
print("\nMissing value percentages for each predictor:")
for pred, pct in missing_percentages.items():
    print(f"{pred}: {pct:.2f}%")

# 11. Create different versions of the cleaned dataset
# Version 1: Only keep rows with Compustat data and no missing predictors
merged_df_strict = merged_df[merged_df['has_compustat']].dropna(subset=predictors)
print("\nStrict dataset (no missing values) shape:", merged_df_strict.shape)


# 13. Show samples of each version
print("\nSample of strict dataset:")
print(merged_df_strict[['cusip', 'MthCalDt', 'MthRet', 'datadate'] + predictors].head())

# 14. Create final X and y datasets (using the imputed version as default)
y_strict = merged_df_strict['MthRet']
X_strict = merged_df_strict[predictors]

print("\nFinal dataset shapes:")
print("Strict: X:", X_strict.shape, "y:", y_strict.shape)

merged_df_strict.head()

# Remove unwanted columns from compustat_df
if 'available_date' in merged_df_strict.columns:
    merged_df_strict.drop(columns=['available_date'])
if 'has_compustat' in merged_df_strict.columns:
    merged_df_strict.drop(columns=['has_compustat'])

total_nans = merged_df_strict.isna().sum().sum()
print(f"Total NaNs in dataset: {total_nans}")

rows_with_nan = merged_df_strict.isna().any(axis=1).sum()
print(f"Number of rows with at least one NaN: {rows_with_nan}")


merged_df_strict.to_csv(os.path.join(folder, 'merged_data_strict.csv'), index=False)


Returns dataset shape: (4725384, 9)
Compustat dataset shape: (3848428, 43)
Returns date range: 1961-01-31 00:00:00 to 2024-12-31 00:00:00
Compustat date range: 1961-03-31 00:00:00 to 2025-04-30 00:00:00
Common CUSIPs: 23064

Merged dataset shape: (4725384, 51)
Merged dataset unique CUSIPs: 54128
Rows with Compustat data: 1597582
Percentage with Compustat data: 33.80851164688415 %

Missing value percentages for each predictor:
epspxy: 66.68%
oiadpy: 67.84%
saley: 69.72%
earnings_growth: 70.36%
revenue_growth: 71.05%
eps_surprise: 66.84%
dividend_change: 89.27%
repurchase_intensity: 74.70%

Strict dataset (no missing values) shape: (493737, 52)

Sample of strict dataset:
           cusip   MthCalDt    MthRet   datadate  epspxy  oiadpy   saley  \
315102  86666510 1973-01-31  0.018182 1973-01-31    0.27   1.382   8.453   
321152  86666510 1973-02-28 -0.080357 1973-02-28    0.27   1.382   8.453   
332791  86666510 1973-04-30 -0.106383 1973-04-30    0.32   1.382  10.704   
338549  86666510 1

Total NaNs in dataset: 317532
Number of rows with at least one NaN: 172865
